## 🧠 Paithon Interactive

With `paithon.interactive` you can **make your model interactive** in no time. 

If you are already familiar with Panel you will notice it combines and extends Panels [interact](https://panel.holoviz.org/user_guide/Interact.html?highlight=interactive#interact), [bind](https://panel.holoviz.org/getting_started/Introduction.html?highlight=bind#composing-new-panels), `panel` etc.

In [1]:
import time

import param
import panel as pn
from paithon import interactive

pn.extension(sizing_mode="stretch_width")

## Single Input and Output

Lets start by defining a very simple model that takes a single value as input and returns a `waw` file.

In [2]:
def audio_model(value):
    return f"https://evolution.voxeo.com/library/audio/prompts/numbers/{value}.wav"

You just need to provide **specific `inputs`** to your model and the outputs will magically be determined.

In [3]:
audio_layout = interactive(audio_model, inputs=pn.widgets.Select(value=0, options=[0, 1, 2, 3, 4]), name="Basic")
audio_layout

Row(name='Basic', sizing_mode='stretch_width')
    [0] Select(options=[0, 1, 2, 3, 4], sizing_mode='stretch_width', value=0)
    [1] ParamMethod(method, sizing_mode='stretch_width')

Lets take another example

In [4]:
def video_model(value):
    return f'<img id="slideshow" style="height:300px;width:100%" src="https://picsum.photos/800/300?image={value}"/>'

In [5]:
video_layout = interactive(video_model, inputs=pn.widgets.RadioButtonGroup(value=1, options=[0, 1, 2, 3, 4]), name="Basic")
video_layout

Row(name='Basic', sizing_mode='stretch_width')
    [0] RadioButtonGroup(options=[0, 1, 2, 3, 4], sizing_mode='stretch_width', value=1)
    [1] ParamMethod(method, sizing_mode='stretch_width')

You can specify **specific parameters on `inputs`**.

In [6]:
videolayout_parameters = pn.Row(
    interactive(video_model, inputs=pn.widgets.IntSlider(value=0, start=0, end=5, ).param.value, default_layout=pn.Column, name="Basic"),
    interactive(video_model, inputs=pn.widgets.IntSlider(value=0, start=0, end=5, ).param.value_throttled, default_layout=pn.Column, name="Basic"),
)
videolayout_parameters

Row(sizing_mode='stretch_width')
    [0] Column(name='Basic', sizing_mode='stretch_width')
        [0] IntSlider(end=5, sizing_mode='stretch_width')
        [1] ParamMethod(method, sizing_mode='stretch_width')
    [1] Column(name='Basic', sizing_mode='stretch_width')
        [0] IntSlider(end=5, sizing_mode='stretch_width')
        [1] ParamMethod(method, sizing_mode='stretch_width')

Try dragging the slider on both examples to see the difference between the `value` and the `value_throttled` parameter

Let provide *specific inputs*

In [7]:
video_layout_custom = interactive(video_model, inputs=pn.widgets.RadioButtonGroup(value=1, options=[0, 1, 2, 3, 4]), outputs=pn.pane.HTML(style={"background": "silver", "padding": "25px"}), name="Basic")
video_layout_custom

Row(name='Basic', sizing_mode='stretch_width')
    [0] RadioButtonGroup(options=[0, 1, 2, 3, 4], sizing_mode='stretch_width', value=1)
    [1] HTML(str, sizing_mode='stretch_width', style={'background': 'silver', ...})

## Multiple inputs and outputs

In [8]:
def two_factor_model(value1, value2):
    return video_model(value1), video_model(value2)

If you model returns a tuple or is a generator it will be assumed that the model has multiple outputs

In [9]:
interactive(
    two_factor_model,
    inputs=[
        pn.widgets.IntSlider(value=0, start=0, end=5), 
        pn.widgets.RadioButtonGroup(value=0, options=[0, 1, 2, 3, 4])
    ]
)

Row(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] IntSlider(end=5, sizing_mode='stretch_width')
        [1] RadioButtonGroup(options=[0, 1, 2, 3, 4], sizing_mode='stretch_width', value=0)
    [1] Column(sizing_mode='stretch_width')
        [0] ParamMethod(method, sizing_mode='stretch_width')
        [1] ParamMethod(method, sizing_mode='stretch_width')

Currently the number of outputs will be determined the first time the model is run. So if your model runs a different number of outputs on each run you can specify a specific number of outputs.

In [10]:
def variable_outputs_model(value):
    return tuple(range(1, value + 1))

interactive(variable_outputs_model, inputs=pn.widgets.IntSlider(value=0, start=0, end=5), num_outputs=5)

Row(sizing_mode='stretch_width')
    [0] IntSlider(end=5, sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_width')
        [0] ParamMethod(method, sizing_mode='stretch_width')
        [1] ParamMethod(method, sizing_mode='stretch_width')
        [2] ParamMethod(method, sizing_mode='stretch_width')
        [3] ParamMethod(method, sizing_mode='stretch_width')
        [4] ParamMethod(method, sizing_mode='stretch_width')

You can even provide some of the outputs

In [12]:
interactive(variable_outputs_model, inputs=pn.widgets.IntSlider(value=0, start=0, end=5), outputs=(None, None, pn.pane.Str(background="lightgray"),), num_outputs=5)

Row(sizing_mode='stretch_width')
    [0] IntSlider(end=5, sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_width')
        [0] ParamMethod(method, sizing_mode='stretch_width')
        [1] ParamMethod(method, sizing_mode='stretch_width')
        [2] Str(None, background='lightgray', sizing_mode='stretch_width')
        [3] ParamMethod(method, sizing_mode='stretch_width')
        [4] ParamMethod(method, sizing_mode='stretch_width')

## Generator Model

You model can even be a *generator* that returns the outputs as they are ready

In [13]:
def generator_model(value):
    for index in range(1, value + 1):
        time.sleep(0.1 * index**2)
        yield video_model(index)

interactive(
    generator_model,
    inputs=pn.widgets.IntSlider(value=0, start=1, end=5,).param.value_throttled,
    num_outputs=5,
    loading_indicator=True,   
)

Row(sizing_mode='stretch_width')
    [0] IntSlider(end=5, sizing_mode='stretch_width', start=1)
    [1] Column(sizing_mode='stretch_width')
        [0] ParamMethod(method, sizing_mode='stretch_width')
        [1] ParamMethod(method, sizing_mode='stretch_width')
        [2] ParamMethod(method, sizing_mode='stretch_width')
        [3] ParamMethod(method, sizing_mode='stretch_width')
        [4] ParamMethod(method, sizing_mode='stretch_width')

You can use this to provide outputs as they become available.

## Generator model with Progress reporting

COMING UP. If you provide a Progress widget and your model is a generator it instead be assumed that it reports intermediate results. Each intermediate result can be a single results, a list or tuple of results or even a generator of results.

## Todo

- Support `throttled` argument
- Use better layout with screenshot and
- provide input_params, output_params
- allow providing instantiated default_layout to `interactive`
- `interactive` to extend interact